In [21]:
import pandas as pd
import seaborn as sns
import numpy as np
from math import sqrt

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 16, 8

from scipy.stats import norm
from scipy import stats

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

In [22]:
df_0 = pd.read_csv('dataset-s0.csv', delimiter=',')
df_0 = df_0.pivot_table('value', ['time'], 'variable')

In [23]:
df_1 = pd.read_csv('dataset-s1.csv', delimiter=',')
df_1 = df_1.pivot_table('value', ['time'], 'variable')

In [24]:
df_2 = pd.read_csv('dataset-o1.csv', delimiter=',')
df_2 = df_2.pivot_table('value', ['time'], 'variable')

In [25]:
df_3 = pd.read_csv('dataset-o2.csv', delimiter=',')
df_3 = df_3.pivot_table('value', ['time'], 'variable')

In [26]:
frames = [df_0, df_1, df_2, df_3]
df = pd.concat(frames)

In [27]:
df=df.dropna(axis=0)

In [28]:
df.drop(columns=["servers", "utility"], axis = 1, inplace=True)

## Training

In [29]:
features = ['activeServers', 'dimmer', 'requestRate']
X = df[features]
y = df['responseTime']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [31]:
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression().fit(X, y)

In [32]:
print(linear_reg.score(X_train, y_train))
print(linear_reg.score(X_test, y_test))

0.22330124551507413
0.08328899782500121


In [33]:
from sklearn.metrics import mean_squared_error
y_pred = linear_reg.predict(X_test)
print(mean_squared_error(y_test, y_pred))

1.0393371731748409


In [34]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
rf_regr = RandomForestRegressor(max_depth=2, random_state=0)
rf_regr.fit(X, y)

RandomForestRegressor(max_depth=2, random_state=0)

In [35]:
print(rf_regr.score(X_train, y_train))
print(rf_regr.score(X_test, y_test))

0.38584428382737657
0.14314132941504232


In [36]:
y_pred = rf_regr.predict(X_test)
print(mean_squared_error(y_test, y_pred))

0.971478542728469


In [37]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X, y)

KNeighborsRegressor(n_neighbors=2)

In [38]:
print(neigh.score(X_train, y_train))
print(neigh.score(X_test, y_test))

0.7052071375890998
0.6989194602309972


In [39]:
y_pred = neigh.predict(X_test)
print(mean_squared_error(y_test, y_pred))

0.34135534138787815


In [42]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import keras
from keras.models import Sequential
from keras.layers import Dense

In [54]:
df.isna().sum()

variable
activeServers    0
dimmer           0
requestRate      0
responseTime     0
dtype: int64

In [55]:
model = Sequential()
model.add(Dense(100, input_dim=3, activation= "relu", input_shape=[len(X_train.keys())]))
model.add(Dense(50, activation= "relu"))
model.add(Dense(0, activation= "relu"))
model.add(Dense(1))

In [56]:
model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])


In [58]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
28/28 [==============================] - 0s 1ms/step - loss: 0.9577 - mean_squared_error: 0.9577
Epoch 2/20
28/28 [==============================] - 0s 2ms/step - loss: 0.9577 - mean_squared_error: 0.9577
Epoch 3/20
28/28 [==============================] - 0s 3ms/step - loss: 0.9577 - mean_squared_error: 0.9577
Epoch 4/20
28/28 [==============================] - 0s 2ms/step - loss: 0.9577 - mean_squared_error: 0.9577
Epoch 5/20
28/28 [==============================] - 0s 1ms/step - loss: 0.9577 - mean_squared_error: 0.9577
Epoch 6/20
28/28 [==============================] - 0s 1ms/step - loss: 0.9577 - mean_squared_error: 0.9577
Epoch 7/20
28/28 [==============================] - 0s 1ms/step - loss: 0.9577 - mean_squared_error: 0.9577
Epoch 8/20
28/28 [==============================] - 0s 1ms/step - loss: 0.9577 - mean_squared_error: 0.9577
Epoch 9/20
28/28 [==============================] - 0s 1ms/step - loss: 0.9577 - mean_squared_error: 0.9577
Epoch 10/20
28/28 [=========

In [59]:
pred_train= model.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train)))

pred= model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred))) 

28/28 [==============================] - 0s 819us/step
1.0584717776220625
12/12 [==============================] - 0s 1ms/step
1.1457947542123688
